In [8]:
# Importing APIs&WebScraping functions
from fetch_crypto_data import fetch_crypto_data
from scrape_bitcoin_news import scrape_bitcoin_news

In [2]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os
load_dotenv()

True

In [10]:
# Database connection parameters
DATABASE_URL = os.getenv('DATABASE_URL')
# Connect to the database
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()


# Create tables if they don't exist
cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_data_py(
    date DATE PRIMARY KEY,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_news_py(
    id SERIAL PRIMARY KEY,
    Date DATE,
    Title TEXT,
    UNIQUE (Date, Title)
)
''')

conn.commit()

In [11]:
# Function to insert data into the bitcoin_data table
def insert_bitcoin_data(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO bitcoin_data_py (date, open, high, low, close, volume)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (date) DO NOTHING
            ''', (index, row['1. open'], row['2. high'], row['3. low'], row['4. close'], row['5. volume']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f"Error inserting bitcoin data: {e}")
        conn.rollback()

In [12]:
# Function to insert data into the bitcoin_news table
def insert_bitcoin_news(conn, df):
    try:
        cursor = conn.cursor()
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO bitcoin_news_py (id, title, date)
                VALUES (%s, %s, %s)
                ON CONFLICT (id) DO NOTHING
            ''', (index, row['Title'], row['Date']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f"Error inserting bitcoin news: {e}")
        conn.rollback()

In [13]:
# Main function to orchestrate the data fetching and insertion
def main():
    # Fetch Bitcoin data
    bitcoin_data = fetch_crypto_data('BTC', 'USD')

    # Scrape Bitcoin news data
    bitcoin_news_df = scrape_bitcoin_news()

    # Connect to the PostgreSQL database
    conn = psycopg2.connect(DATABASE_URL)

    if conn:
        # Insert data into the database
        insert_bitcoin_data(conn, bitcoin_data)
        insert_bitcoin_news(conn, bitcoin_news_df)

        # Close the database connection
        conn.close()
    else:
        print("Failed to connect to the database.")

if __name__ == '__main__':
    main()

### Create tables for AWS functions

In [3]:
# Database connection parameters
DATABASE_URL = os.getenv('DATABASE_URL')
# Connect to the database
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()


# Create tables if they don't exist
cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_data_aws(
    date DATE PRIMARY KEY,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_news_aws(
    Date DATE PRIMARY KEY,
    Title TEXT
    )
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_news_sentiment(
    date DATE PRIMARY KEY,
    title TEXT,
    sentiment TEXT  
    )
''')


cur.execute('''
CREATE TABLE IF NOT EXISTS BitcoinNews_SentimentScore(
    date DATE,
    title TEXT PRIMARY KEY,
    neutral FLOAT,
    negative FLOAT,
    positive FLOAT
    )
''')

conn.commit()

In [22]:
# Database connection parameters
DATABASE_URL = os.getenv('DATABASE_URL')
# Connect to the database
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()

# DELETE tables 
cur.execute('''DROP TABLE bitcoin_data3;''')

conn.commit()